In [1]:
#Importar Bibliotecas
import numpy as np
import pandas as pd
import pyodbc

#Variáveis de conexão SQL
server = 'PRIMNO3' 
database = 'cobmailing' 
username = '######' 
password = '######'  
#estabelece a conexão SQL
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';trusted_connection=true')
                      #UID='+username+';PWD='+ password) #Usar essa linha caso for usar autenticação do SQL
cursor = cnxn.cursor()



In [2]:
#carregar o arquivo excel
df1 = pd.read_excel('Negociacao_Cliente.xlsx', engine="openpyxl")
print('Arquivo carregado...')

C:\Users\julio.lima\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Arquivo carregado...


In [3]:
#descreve os tipos de dados das colunas
print(df1.describe())

        COD_CLIENTE     ID_PESSOA      ID_CONTR     QTD_NEGOC  VLR_ORIGINAL  \
count  2.372000e+04  2.372000e+04  2.372000e+04  23720.000000  23720.000000   
mean   1.190222e+06  3.098357e+06  8.031129e+05      1.794351   1395.915494   
std    4.454923e+05  2.398905e+06  3.431832e+05      0.956291   1803.102134   
min    1.206000e+03  2.570000e+02  6.600000e+01      1.000000      0.780000   
25%    9.853275e+05  9.588440e+05  5.388672e+05      1.000000    488.335000   
50%    1.305828e+06  2.728951e+06  9.261090e+05      2.000000    909.440000   
75%    1.543630e+06  4.380311e+06  1.100282e+06      2.000000   1681.147500   
max    1.689809e+06  7.865113e+06  1.201688e+06     13.000000  83880.070000   

       VLR_NEGOCIADO       USUARIO  TEMPO_CARTEIRA     QTD_VIDAS  
count    3915.000000  23720.000000    23546.000000  23720.000000  
mean     1048.147706   2827.614460     3192.877049      3.024368  
std      1413.982630    404.522639     2369.865708      2.969922  
min        55.180000

In [4]:
#Conversão de dados para Inteiro
df1['NUM_CARTEIRA'] = df1['NUM_CARTEIRA'].apply(lambda x: str(x).replace("*",""))
print('Conversão realizada...')
df1['NUM_CARTEIRA'] = df1['NUM_CARTEIRA'].astype('int64')

Conversão realizada...


In [5]:
df1.info()
#dt_promess = dt_filter.assign(NUM_CARTEIRA = (dt_filter1['NUM_CARTEIRA']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23720 entries, 0 to 23719
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   NUM_CARTEIRA              23720 non-null  int64         
 1   NOM_CLIENTE               23720 non-null  object        
 2   COD_CLIENTE               23720 non-null  int64         
 3   ID_PESSOA                 23720 non-null  int64         
 4   ID_CONTR                  23720 non-null  int64         
 5   DT_CAD_CLIENTE            23720 non-null  datetime64[ns]
 6   DT_GERACAO                23720 non-null  datetime64[ns]
 7   DT_VENC_TITULO_MAIS_NOVO  23720 non-null  datetime64[ns]
 8   DT_VENC_ENTRADA           23720 non-null  datetime64[ns]
 9   DT_VENC_ULTIMA_PARCELA    23720 non-null  datetime64[ns]
 10  QTD_NEGOC                 23720 non-null  int64         
 11  VLR_ORIGINAL              23720 non-null  float64       
 12  VLR_NEGOCIADO     

In [6]:
#Executa a consulta SQL para carregar os dados do mailing
texto = '"'
query = "Select NUM_CARTEIRA,NUM_CNPJ_CPF,REPLACE(TEL1,'{}','') as TEL1, REPLACE(TEL2,'{}','') as TEL2, REPLACE(TEL3,'{}','') as TEL3, REPLACE(TEL4,'{}','') as TEL4, REPLACE(TEL5,'{}','') as TEL5, REPLACE(TEL6,'{}','') as TEL6 from COBMailing.dbo.Unimed_Mailing_PME_Individual with (nolock)".format(texto,texto,texto,texto,texto,texto)
df2 = pd.read_sql(query, cnxn)

#print('Consulta executada com sucesso!')

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572202 entries, 0 to 572201
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   NUM_CARTEIRA  572202 non-null  int64 
 1   NUM_CNPJ_CPF  572202 non-null  int64 
 2   TEL1          572202 non-null  object
 3   TEL2          342383 non-null  object
 4   TEL3          181221 non-null  object
 5   TEL4          91713 non-null   object
 6   TEL5          46168 non-null   object
 7   TEL6          24275 non-null   object
dtypes: int64(2), object(6)
memory usage: 34.9+ MB


In [8]:
#Realiza a união de filtragem das duas dataframes

dt_completa = pd.merge(df1, df2, on = 'NUM_CARTEIRA')

dt_completa = dt_completa

dt_final = dt_completa.drop_duplicates(subset=['NUM_CARTEIRA', 'DT_VENC_ENTRADA'])       #dt_completa[['NUM_CARTEIRA','DT_VENC_ENTRADA','VLR_ORIGINAL','VLR_NEGOCIADO','TEL1','TEL2','TEL3','TEL4','TEL5','TEL6',
                           #'TEL7','TEL8','TEL9']]

dt_final = dt_final[['NUM_CARTEIRA','NUM_CNPJ_CPF','DT_VENC_ENTRADA','VLR_ORIGINAL','VLR_NEGOCIADO','TEL1','TEL2','TEL3','TEL4',
                     'TEL5','TEL6']]

In [9]:
#Exibição de parcial da dataframe
dt_final.head(100)
#Substitui os campos null por 0
dt_final = dt_final.replace(np.nan, 0)

In [10]:
#escreve a dataframe no SQL Server
from sqlalchemy import create_engine, event
from urllib.parse import quote_plus

conn =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';trusted_connection=true'
quoted = quote_plus(conn)
new_con = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(new_con)


#dt_final.to_sql('Unimed_Acordos_Cliente', engine, if_exists='append', chunksize = None)
#dt_completa.to_excel('teste.xlsx')
#print('File saved sucessfuly...')

In [11]:
#dt_final.to_sql('Unimed_Acordos_Cliente', engine, if_exists='append', chunksize = None)

In [12]:
errLinhas = 0
for x, y in dt_final.iterrows():
    try: 
        pd.read_sql("insert into Unimed_Acordos_Cliente values ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(x,y[0],y[1],y[2],y[3],y[4],y[5],y[6],y[7],y[8],y[9],y[10]), cnxn)
    except TypeError:
        errLinhas += 1
        
    #print (y[4])
    
    #print("insert into Unimed_Acordos_Cliente values ('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(x,y[0],y[1],y[2],y[3],y[4],y[5],y[6],y[7],y[8],y[9],y[10]))
print('Total de linhas com erro: {}'.format(errLinhas))   
    
    
    

Total de linhas com erro: 19774
